In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer_all = Timer()
timer_all.reset()

In [3]:
timer.reset()
tb = TbPacientes()
# tb.update()
timer.stop()

1.3753211498260498

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

1.1250622272491455

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (583786, 24)
casos tb_pacientes: (583786, 48)


In [6]:
casosc['identificacao'] = casosc['identificacao'].apply(int)
casosc = casosc.sort_values('identificacao')
casosc.head()

,ordem,identificacao,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_dt_diag
910,911,1,4108304,9.0,ABBAS MOHAMAD MESSELMANI,M,41,FOZ DO IGUACU,FOZ DO IGUACU,09,LACEN,2020-04-17,2020-04-18,2020-04-11,NAO,NaT,NaN,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU,ABBASMOHAMADMESSELMANI20200417000000
27968,27969,2,4117107,14.0,ABDIEL DE LIMA GONCALVES,M,44,NOVA LONDRINA,NOVA LONDRINA,14,IBMP,2020-07-03,2020-07-04,2020-06-22,NAO,NaT,NaN,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA,ABDIELDELIMAGONCALVES20200703000000
5513,5514,3,4127700,20.0,ABDOU INDIAYE,M,31,TOLEDO,TOLEDO,20,IBMP,2020-06-03,2020-06-04,2020-05-29,NAO,NaT,NaN,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO,ABDOUINDIAYE20200603000000
6612,6613,4,4112009,3.0,ABEDIEL DA SILVA,M,38,JAGUARIAIVA,JAGUARIAIVA,03,COVID-19 IGG/IGM ECO TESTE,2020-06-05,2020-06-06,NaT,NAO,NaT,NaN,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA,ABEDIELDASILVA20200605000000
10589,10590,5,4125506,2.0,ABEGAIL DOMINGUES,F,59,SAO JOSE DOS PINHAIS,SAO JOSE DOS PINHAIS,02,IBMP,2020-06-16,2020-06-17,2020-06-09,NAO,NaT,NaN,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES20200616000000


In [7]:
casost['identificacao'] = casost['identificacao'].apply(int)
casost = casost.sort_values('identificacao')
casost.head()

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend
0,1,4108304,4108304,ABBAS MOHAMAD MESSELMANI,M,41,FOZ DO IGUACU,FOZ DO IGUACU,LACEN,2020-04-17,2020-04-18,2020-04-11,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-28,REGIONAL DE SAÚDE,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23.0,41,NaT,NaT,42,40,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU
1,2,4117107,4117107,ABDIEL DE LIMA GONCALVES,M,44,NOVA LONDRINA,NOVA LONDRINA,IBMP,2020-07-03,2020-07-04,2020-06-22,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-07-30,NOTIFICA COVID,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,44,NaT,2020-08-05,45,43,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA
2,3,4127700,4127700,ABDOU INDIAYE,M,31,TOLEDO,TOLEDO,IBMP,2020-06-03,2020-06-04,2020-05-29,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-05,paciente,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,31,NaT,NaT,32,30,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO
3,4,4112009,4112009,ABEDIEL DA SILVA,M,38,JAGUARIAIVA,JAGUARIAIVA,COVID-19 IgG/IgM ECO Teste,2020-06-05,2020-06-06,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-17,SMS Jaguariaíva,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,6.0,38,NaT,NaT,39,37,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA
4,5,4125506,4125506,ABEGAIL DOMINGUES,F,59,SAO JOSE DOS PINHAIS,SAO JOSE DOS PINHAIS,IBMP,2020-06-16,2020-06-17,2020-06-09,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,NaT,NaN,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,59,NaT,NaT,60,58,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS


In [8]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [9]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [10]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)
if (len(casos_confirmados_nao_tb_pacientes) > 0) or (len(tb_pacientes_nao_casos_confirmados) > 0):
    raise Exception("diferentes")

casos_confirmados_nao_tb_pacientes: (0, 24)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [11]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [12]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [13]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))
print(all(casosc['nome'].values == casost['nome'].values))
print(all(casosc['mun_resid'].values == casost['mun_resid'].values))
print(all(casosc['mun_atend'].values == casost['mun_atend'].values))

True
True
False
True
False
False


In [14]:
timer.reset()

# sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
# sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
# i = 0
# for (ibge_cc, df_ibge_cc), (ibge_tb, df_ibge_tb) in zip(casosc.groupby('ibge7'),casost.groupby('ibge_res_pr')):
#     print(i,end=' ')
#     i+=1
#     if ibge_cc != ibge_tb:
#         raise Exception(ibge_cc,ibge_tb)

#     df_ibge_cc = df_ibge_cc.sort_values('nome_hash')
#     df_ibge_tb = df_ibge_tb.sort_values('nome_hash')

#     duplicados_ibge_cc = df_ibge_cc.loc[ df_ibge_cc.duplicated('nome_hash', keep=False) ]
#     duplicados_ibge_tb = df_ibge_tb.loc[ df_ibge_tb.duplicated('nome_hash', keep=False) ]

#     sorted_casosc = sorted_casosc.append(df_ibge_cc.loc[~df_ibge_cc.index.isin(duplicados_ibge_cc.index)])
#     sorted_casost = sorted_casost.append(df_ibge_tb.loc[~df_ibge_tb.index.isin(duplicados_ibge_tb.index)])

#     for (nome_cc, df_nome_cc), (nome_tb, df_nome_tb) in zip(duplicados_ibge_cc.groupby('nome_hash'),duplicados_ibge_tb.groupby('nome_hash')):
#         if nome_cc != nome_tb:
#             raise Exception(nome_cc,nome_tb)

#         sorted_casosc = sorted_casosc.append(df_nome_cc.sort_values('idade'))
#         sorted_casost = sorted_casost.append(df_nome_tb.sort_values('idade'))


# print(f"casos casos_confirmados: {sorted_casosc.shape}")
# print(f"casos tb_pacientes: {casost.shape}")
# print(timer.stop())
# casosc = sorted_casosc
# casost = sorted_casost

In [15]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))
print(all(casosc['nome'].values == casost['nome'].values))
print(all(casosc['mun_resid'].values == casost['mun_resid'].values))
print(all(casosc['mun_atend'].values == casost['mun_atend'].values))

True
True
False
True
False
False


In [16]:
casosc = casosc.rename(columns={'ibge7':'ibge_resid','nome':'paciente','dt_diag':'data_diagnostico','comunicacao':'data_comunicacao','is':'data_1o_sintomas'})

casosc['ordem'] = casosc['ordem'].apply(int)
casosc['identificacao'] = casost['identificacao'].apply(int).values.tolist()
casosc['ibge_atend'] = casost['ibge_atend_pr'].apply(int).values.tolist()
casosc['ibge_resid'] = casosc['ibge_resid'].apply(int)

casosc.loc[casosc['rs'].isnull(),'rs'] = 99
casosc['rs'] = casosc['rs'].apply(int)

casosc['recuperado'] = [ 1 if recuperado=="Recuperado" else 0 for recuperado in casost['status'] ]
casosc['obito'] = [ 1 if obito=="SIM" else 0 for obito in casosc['obito'] ]
casosc.loc[casosc['obito']==1, 'data_com_obito'] = casost.loc[casost['obito']=="SIM", 'dt_com_obito']

casosc['ativo'] = 0
casosc.loc[(casosc['recuperado']==0) & (casosc['obito']==0 ), 'ativo'] = 1
casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'] = 0

casosc = casosc[['ordem', 'identificacao', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'obito', 'data_obito', 'data_com_obito', 'recuperado', 'ativo', 'excluir']].sort_values('ordem')

In [17]:
writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,'geral',index=None)
worksheet = writer.sheets[f"geral"]
auto_fit_columns(worksheet,casosc)

writer.save()
writer.close()

In [18]:
print(timedelta(seconds=timer_all.stop()))

0:03:05.500824


In [19]:
casosc

,ordem,identificacao,ibge_resid,ibge_atend,paciente,sexo,idade,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,obito,data_obito,data_com_obito,recuperado,ativo,excluir
0,1,8838,4105508,4105508,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,13,LACEN,2020-03-12,2020-03-12,NaT,0,NaT,NaT,1,0,NaN
1,2,10932,4106902,4106902,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-07,0,NaT,NaT,1,0,NaN
2,3,10933,4106902,4106902,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,2,LACEN,2020-03-12,2020-03-12,2020-03-08,0,NaT,NaT,1,0,NaN
3,4,11470,4106902,4106902,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-16,2020-03-16,2020-03-11,0,NaT,NaT,1,0,NaN
4,5,11492,4106902,4106902,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,2,GENOPRIMER,2020-03-18,2020-03-18,2020-03-15,0,NaT,NaT,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583781,583782,665532,4118402,4118402,ZILMA PAULINA DA SILVA,F,73,PARANAVAI,PARANAVAI,14,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-11,2021-02-12,2021-02-06,0,NaT,NaT,0,1,NaN
583782,583783,665533,4113700,4113700,ZITA TOTTENE,F,74,LONDRINA,LONDRINA,17,COVID-19 BIOLOGIA MOLECULAR,2021-02-11,2021-02-12,2021-02-08,0,NaT,NaT,0,1,NaN
583783,583784,665534,4116703,4116703,ZULEIDE DAL ROVERE DOS SANTOS,F,51,NOVA AURORA,NOVA AURORA,10,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-08,2021-02-12,2021-02-05,0,NaT,NaT,0,1,NaN
583784,583785,665535,4118501,4118501,ZULMIR CICHOCKI,M,65,PATO BRANCO,PATO BRANCO,7,COVID-19 BIOLOGIA MOLECULAR,2021-02-10,2021-02-12,2021-02-06,0,NaT,NaT,0,1,NaN


In [20]:
casost

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,nome_hash
0,1,4108304,4108304,ABBAS MOHAMAD MESSELMANI,M,41,FOZ DO IGUACU,FOZ DO IGUACU,LACEN,2020-04-17,2020-04-18,2020-04-11,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-28,REGIONAL DE SAÚDE,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23.0,41,NaT,NaT,42,40,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU,ABBASMOHAMADMESSELMANI41FOZDOIGUACU,ABBASMOHAMADMESSELMANI40FOZDOIGUACU,ABBASMOHAMADMESSELMANI42FOZDOIGUACU,ABBASMOHAMADMESSELMANI
1,2,4117107,4117107,ABDIEL DE LIMA GONCALVES,M,44,NOVA LONDRINA,NOVA LONDRINA,IBMP,2020-07-03,2020-07-04,2020-06-22,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-07-30,NOTIFICA COVID,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,44,NaT,2020-08-05,45,43,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA,ABDIELDELIMAGONCALVES44NOVALONDRINA,ABDIELDELIMAGONCALVES43NOVALONDRINA,ABDIELDELIMAGONCALVES45NOVALONDRINA,ABDIELDELIMAGONCALVES
2,3,4127700,4127700,ABDOU INDIAYE,M,31,TOLEDO,TOLEDO,IBMP,2020-06-03,2020-06-04,2020-05-29,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-05,paciente,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,31,NaT,NaT,32,30,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO,ABDOUINDIAYE31TOLEDO,ABDOUINDIAYE30TOLEDO,ABDOUINDIAYE32TOLEDO,ABDOUINDIAYE
3,4,4112009,4112009,ABEDIEL DA SILVA,M,38,JAGUARIAIVA,JAGUARIAIVA,COVID-19 IgG/IgM ECO Teste,2020-06-05,2020-06-06,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,2020-06-17,SMS Jaguariaíva,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,6.0,38,NaT,NaT,39,37,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA,ABEDIELDASILVA38JAGUARIAIVA,ABEDIELDASILVA37JAGUARIAIVA,ABEDIELDASILVA39JAGUARIAIVA,ABEDIELDASILVA
4,5,4125506,4125506,ABEGAIL DOMINGUES,F,59,SAO JOSE DOS PINHAIS,SAO JOSE DOS PINHAIS,IBMP,2020-06-16,2020-06-17,2020-06-09,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,NaT,NaN,Recuperado,NaN,NaN,NaN,NaT,NaN,NaN,NaN,19.0,59,NaT,NaT,60,58,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES59SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES58SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES60SAOJOSEDOSPINHAIS,ABEGAILDOMINGUES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583781,665532,4118402,4118402,ZILMA PAULINA DA SILVA,F,73,PARANAVAI,PARANAVAI,Teste Rápido Antígeno (Imunocromatografia),2021-02-11,2021-02-12,2021-02-06,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,192.0,73,NaT,NaT,74,72,ZILMAPAULINADASILVA73PARANAVAI,ZILMAPAULINADASILVA72PARANAVAI,ZILMAPAULINADASILVA74PARANAVAI,ZILMAPAULINADASILVA73PARANAVAI,ZILMAPAULINADASILVA72PARANAVAI,ZILMAPAULINADASILVA74PARANAVAI,ZILMAPAULINADASILVA
583782,665533,4113700,4113700,ZITA TOTTENE,F,74,LONDRINA,LONDRINA,"COVID-19, Biologia Molecular",2021-02-11,2021-02-12,2021-02-08,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NAO,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130.0,74,NaT,NaT,75,73,ZITATOTTENE74LONDRINA,ZITATOTTENE73LONDRINA,ZITATOTTENE75LONDRINA,ZITATOTTENE74LONDRINA,ZITATOTTENE73LONDRINA,ZITATOTTENE75LONDRINA,ZITATOTTENE
583783,665534,4116703,4116703,ZULEIDE DAL ROVERE DOS SANTOS,F,51,